In [1]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
%matplotlib inline

In [2]:
start = datetime.datetime(2022,8,12)
end = datetime.datetime(2022,8,12)

In [3]:
share = data.DataReader('^NSEI', 'yahoo', start, end)

In [4]:
returns = pd.DataFrame()

In [5]:
returns['High'] =share['High'].pct_change()*100
returns['Low'] =share['Low'].pct_change()*100
returns['Open'] =share['Open'].pct_change()*100
returns['Close'] =share['Close'].pct_change()*100
returns['Volume'] =share['Volume'].pct_change()*100

In [6]:
returns.tail()

,High,Low,Open,Close,Volume
Date,,,,,
2022-08-12,NaN,NaN,NaN,NaN,NaN


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# Optional Plotly Method Imports
import plotly
import cufflinks as cf
cf.go_offline()
returns[['Open', 'High', 'Low', 'Close']].iplot(kind='bar')

In [8]:
share['Volume'].iplot(kind='bar')

In [9]:
share[['Open', 'High', 'Low','Close']].iplot('candle')

In [10]:
returns['Volume'].iplot(kind='bar')

In [11]:
nifty = pd.read_excel('INDEX.xlsx','NIFTY50')
banknifty=pd.read_excel('INDEX.xlsx','BANKNIFTY')

In [12]:
nifty.head()

,Ticker,Weightage
0,ADANIPORTS.NS,0.73
1,AXISBANK.NS,2.54
2,ASIANPAINT.NS,2.00
3,BAJAJFINSV.NS,1.21
4,BAJAJ-AUTO.NS,0.68


In [13]:
banknifty.head()

,Ticker,Weightage
0,AXISBANK.NS,10.46
1,AUBANK.NS,1.16
2,BANDHANBNK.NS,1.08
3,FEDERALBNK.NS,0.91
4,HDFCBANK.NS,32.70


In [14]:
nifty.set_index('Ticker',inplace=True)
banknifty.set_index('Ticker',inplace=True)

In [15]:
banknifty.head()

,Weightage
Ticker,
AXISBANK.NS,10.46
AUBANK.NS,1.16
BANDHANBNK.NS,1.08
FEDERALBNK.NS,0.91
HDFCBANK.NS,32.70


In [16]:
tracking_stocks=pd.DataFrame()
tracking_stocks.index=nifty.index
positive=0
negative=0
neutral=0
tracking_stocks['Change']=0
tracking_stocks['Volume']=0
tracking_stocks['Value']=0
convert_dict = {'Change':float,
                'Volume': float,
                'Value':float
               }
tracking_stocks=tracking_stocks.astype(convert_dict)
nifty_50=pd.DataFrame()
range_date = pd.date_range(start,end)
final_nifty=pd.DataFrame()
final_nifty=range_date.to_frame().drop(columns=0,axis=1)
final_nifty['Relative Value']=0
final_nifty['Total Volume']=0
final_nifty['Sigma Value']=0
convert_dict = {'Relative Value':float,
                'Total Volume': float,
                'Sigma Value':float
               }
final_nifty=final_nifty.astype(convert_dict)
final_nifty.head()

,Relative Value,Total Volume,Sigma Value
2022-08-12,0.0,0.0,0.0


In [17]:
final_banknifty=pd.DataFrame()
nifty_bank=pd.DataFrame()
final_banknifty=range_date.to_frame().drop(columns=0,axis=1)
final_banknifty['Relative Value']=0
final_banknifty['Total Volume']=0
final_banknifty['Sigma Value']=0
final_banknifty.head()

,Relative Value,Total Volume,Sigma Value
2022-08-12,0,0,0


In [18]:
for stock in list(nifty.index):
    nifty_share=pd.DataFrame()
    nifty_share = data.DataReader(stock, 'yahoo', start, end)    
    nifty_share['Change']=((nifty_share['Close']-nifty_share['Open'])/nifty_share['Open'])*100
    final_nifty = final_nifty.merge(nifty_share,left_index=True, right_index=True,how='left')
    final_nifty['Change'].fillna(0,inplace = True)
    final_nifty['Volume'].fillna(0,inplace=True)
    final_nifty['Sigma Value'].fillna(0,inplace=True)
    final_nifty['Change']=final_nifty['Change']*float(nifty.loc[stock])
    final_nifty['Value']=final_nifty['Change']*final_nifty['Volume']
    final_nifty['Sigma Value']= final_nifty['Sigma Value']+final_nifty['Value']
    final_nifty['Relative Value']=final_nifty['Relative Value']+final_nifty['Change']
    final_nifty['Total Volume']=final_nifty['Total Volume']+final_nifty['Volume']
    tracking_stocks.loc[stock]['Change']=(nifty_share.loc[end]['Change'])
    tracking_stocks.loc[stock]['Volume']=nifty_share.loc[end]['Volume']
    tracking_stocks.loc[stock]['Value']=(nifty_share.loc[end]['Change']*nifty_share.loc[end]['Volume'])/1000
    if tracking_stocks.loc[stock]['Change']>0:
        positive=positive+1
    elif tracking_stocks.loc[stock]['Change']==0:
        neutral=neutral+1
    else:
        negative=negative+1
    final_nifty.drop(columns=['Open','High','Low','Close','Volume','Adj Close','Change','Value'],axis=1,inplace=True)


In [19]:
final_nifty['Total']=final_nifty['Relative Value']+final_nifty['Total Volume']
final_nifty=final_nifty[final_nifty['Total']>0]
final_nifty['Sigma Value']=final_nifty['Sigma Value']/100000
final_nifty.drop(columns=['Total'],axis=1,inplace=True)

In [20]:
print('Positive',positive)
print('Negative',negative)
print('Neutral',neutral)

Positive 27
Negative 23
Neutral 0


In [21]:
tracking_stocks.sort_values(by=['Change'], ascending=False)

,Change,Volume,Value
Ticker,,,
ONGC.NS,3.958180,40876891.0,161798.100763
NTPC.NS,3.122969,17998843.0,56209.826313
TATASTEEL.NS,3.017835,95942333.0,289538.176599
UPL.NS,2.721357,3744532.0,10190.209664
SBILIFE.NS,2.473122,1572048.0,3887.866759
POWERGRID.NS,2.264575,9959297.0,22553.578584
BPCL.NS,1.690266,2642970.0,4467.321750
RELIANCE.NS,1.648456,5838244.0,9624.087134
COALINDIA.NS,1.210043,8861089.0,10722.297610


In [22]:
final_nifty['Relative Value'].iplot(kind='bar')

In [23]:
final_nifty['Total Volume'].iplot(kind='bar')

In [24]:
final_nifty['Sigma Value'].iplot(kind='bar')

In [25]:
final_nifty

,Relative Value,Total Volume,Sigma Value
2022-08-12,21.823724,303872483.0,7465.060825


In [26]:
tracking_stocks=pd.DataFrame()
tracking_stocks.index=nifty.index
positive=0
negative=0
neutral=0
tracking_stocks['Change']=0
tracking_stocks['Volume']=0
tracking_stocks['Value']=0
convert_dict = {'Change':float,
                'Volume': float,
                'Value':float
               }
tracking_stocks=tracking_stocks.astype(convert_dict)
nifty_50=pd.DataFrame()
range_date = pd.date_range(start,end)
final_nifty=pd.DataFrame()
final_nifty=range_date.to_frame().drop(columns=0,axis=1)
final_nifty['Relative Value']=0
final_nifty['Total Volume']=0
final_nifty['Sigma Value']=0
convert_dict = {'Relative Value':float,
                'Total Volume': float,
                'Sigma Value':float
               }
final_nifty=final_nifty.astype(convert_dict)
final_nifty.head()

,Relative Value,Total Volume,Sigma Value
2022-08-12,0.0,0.0,0.0


In [27]:
for stock in list(nifty.index):
    nifty_share=pd.DataFrame()
    nifty_share = data.DataReader(stock, 'yahoo', start, end)    
    nifty_share['Change']=(nifty_share['Close'].pct_change())*100
    final_nifty = final_nifty.merge(nifty_share,left_index=True, right_index=True,how='left')
    final_nifty['Change'].fillna(0,inplace = True)
    final_nifty['Volume'].fillna(0,inplace=True)
    final_nifty['Sigma Value'].fillna(0,inplace=True)
    final_nifty['Change']=final_nifty['Change']*float(nifty.loc[stock])
    final_nifty['Value']=final_nifty['Change']*final_nifty['Volume']
    final_nifty['Sigma Value']= final_nifty['Sigma Value']+final_nifty['Value']
    final_nifty['Relative Value']=final_nifty['Relative Value']+final_nifty['Change']
    final_nifty['Total Volume']=final_nifty['Total Volume']+final_nifty['Volume']
    tracking_stocks.loc[stock]['Change']=(nifty_share.loc[end]['Change'])
    tracking_stocks.loc[stock]['Volume']=nifty_share.loc[end]['Volume']
    tracking_stocks.loc[stock]['Value']=(nifty_share.loc[end]['Change']*nifty_share.loc[end]['Volume'])/1000
    if tracking_stocks.loc[stock]['Change']>0:
        positive=positive+1
    elif tracking_stocks.loc[stock]['Change']==0:
        neutral=neutral+1
    else:
        negative=negative+1
    final_nifty.drop(columns=['Open','High','Low','Close','Volume','Adj Close','Change','Value'],axis=1,inplace=True)


In [28]:
final_nifty['Total']=final_nifty['Relative Value']+final_nifty['Total Volume']
final_nifty=final_nifty[final_nifty['Total']>0]
final_nifty['Sigma Value']=final_nifty['Sigma Value']/100000
final_nifty.drop(columns=['Total'],axis=1,inplace=True)

In [29]:
print('Positive',positive)
print('Negative',negative)
print('Neutral',neutral)

Positive 0
Negative 50
Neutral 0


In [30]:
tracking_stocks.sort_values(by=['Change'], ascending=True)

,Change,Volume,Value
Ticker,,,
ADANIPORTS.NS,NaN,2281608.0,NaN
AXISBANK.NS,NaN,4614341.0,NaN
ASIANPAINT.NS,NaN,548472.0,NaN
BAJAJFINSV.NS,NaN,205251.0,NaN
BAJAJ-AUTO.NS,NaN,217139.0,NaN
BAJFINANCE.NS,NaN,1036941.0,NaN
BRITANNIA.NS,NaN,228587.0,NaN
BHARTIARTL.NS,NaN,4540159.0,NaN
BPCL.NS,NaN,2642970.0,NaN


In [31]:
final_nifty['Relative Value'].iplot(kind='bar')

In [32]:
final_nifty['Total Volume'].iplot(kind='bar')

In [33]:
final_nifty['Sigma Value'].iplot('bar')

In [34]:
stock

'WIPRO.NS'

In [35]:
nifty_share

,High,Low,Open,Close,Volume,Adj Close,Change
Date,,,,,,,
2022-08-12,438.5,435,437.100006,436.100006,3546199,436.100006,NaN
